In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [21]:
mean_sale_price = pd.read_csv(r'Mean Sales Price of Houses Sold.csv')
median_sale_price = pd.read_csv(r'Median Sales Price of Houses Sold.csv')
house_price_index = pd.read_csv(r'All-Transactions House Price Index.csv')
median_income = pd.read_csv(r'Median Household Income.csv')

print("Mean sales price: \n" + str(mean_sale_price.head()))
print("Median sales price: \n" + str(median_sale_price.head()))
print("House price index: \n" + str(house_price_index.head()))
print("Median household income: \n" + str(median_income.head()))

Mean sales price: 
  observation_date  ASPUS
0       1963-01-01  19300
1       1963-04-01  19400
2       1963-07-01  19200
3       1963-10-01  19600
4       1964-01-01  19600
Median sales price: 
  observation_date  MSPUS
0       1963-01-01  17800
1       1963-04-01  18000
2       1963-07-01  17900
3       1963-10-01  18500
4       1964-01-01  18500
House price index: 
  observation_date  USSTHPI_NBD19750101
0       1975-01-01            100.00000
1       1975-04-01            101.55026
2       1975-07-01            102.31705
3       1975-10-01            103.75063
4       1976-01-01            104.83414
Median household income: 
  observation_date  MEHOINUSA646N
0       1984-01-01          22420
1       1985-01-01          23620
2       1986-01-01          24900
3       1987-01-01          26060
4       1988-01-01          27230
